# Import Libraries

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import os

# Define Source

In [2]:
link = 'https://autismcanada.org/autism-explained/'

# Read 100 questions list

In [3]:
file = 'Excel/' + os.listdir('Excel')[-1]
df = pd.read_excel(file,engine='openpyxl')

df[link]=np.nan
df.head()

,Question,https://autismnavigator.com/what-is-autism/,https://www.autism.org/is-it-autism/,https://www.autism.org/advice-for-parents/,https://www.autism.org/symptoms-of-autism/,https://www.autism.org/autism-prognosis/,https://autismcanada.org/autism-explained/
0,What are the Autism Spectrum Disorders (ASD)?,Autism spectrum disorder (ASD) is a neurodevel...,NaN,NaN,NaN,NaN,NaN
1,How is ASD diagnosed?,Diagnosing ASD can be difficult because there ...,NaN,Possible symptoms of autism may be observed by...,NaN,NaN,NaN
2,What are early red flags of ASD in toddlers?,The diagnostic features of ASD can be easy to ...,NaN,NaN,NaN,NaN,NaN
3,What are 16 early signs of autism by 16 months?,The early signs of autism are easy to miss. Au...,NaN,Autism develops sometime during pregnancy and ...,NaN,NaN,NaN
4,What is Autism?,NaN,Autism Spectrum Disorder (ASD) is a developmen...,Autism is a developmental disability that typi...,NaN,Autism is a neurodevelopmental disorder that b...,NaN


# Scrape QA pairs from website

In [4]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(link)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\aparg\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [5]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [6]:
x = soup.find('div',{'class':'elementor-text-editor elementor-clearfix'}).text
x

'\nWhat is Autism?Autism Spectrum Disorder, or autism, is a neurodevelopmental disorder that impacts brain development.\xa0 The result is that most individuals experience communication problems, difficulty with social interactions and a tendency to repeat specific patterns of behaviour. They may also have a markedly restricted range of activities and interests.Autism Spectrum Disorder (ASD) is typically accompanied by co-occurring medical conditions such as epilepsy, sleep disorders, gastrointestinal (gut) abnormalities and immune dysregulation. Mental health issues such as anxiety and depression are common. Any of these conditions may severely impact an individual’s quality of life.The term “spectrum” refers to a range or continuum of severity or of developmental impairment. Children and adults with ASDs may have particular characteristics in common but the condition covers a wide spectrum, with individual differences in the following:Number and particular kinds of symptomsSeverity: M

In [11]:
x = x.split('?')

In [18]:
l1 = [x[0]]
k = [x[1]]

In [19]:
l1

['\nWhat is Autism']

# Check which questions are similar

In [20]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('autism spectrum disorder','asd',sent) # Compress term
    sent = re.sub('autism','asd',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [21]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [22]:
def sim(li):
    for c1,i in enumerate(li):
        i2 = clean_text(i)
        list1 = i2.split()

        temp1 = 0.0
        temp2 = ''
        temp3 = ''
        temp4 = 0
        temp5 = 0

        for c2,j in enumerate(df['Question']):
            j2 = clean_text(j)
            list2 = j2.split()

            sim = jaccard_similarity(list1, list2)

            if(sim>temp1):
                temp1 = sim
                temp2 = j
                temp3 = i
                temp4 = c2
                temp5 = c1

        if(temp1>0):
            print('Website --> ',temp3,'(Index {})'.format(temp5))
            print('100 questions list --> ',temp2,'(Index {})'.format(temp4))
            print(temp1)
            print('*'*100)

In [23]:
sim(l1)

Website -->  
What is Autism (Index 0)
100 questions list -->  What is Autism? (Index 4)
1.0
****************************************************************************************************


In [ ]:
# Website, 100Q
# 0, 4

In [25]:
df.iloc[4,-1]=k[0]

In [34]:
df.iloc[1,-1]='https://autismcanada.org/autism-explained/diagnosis/diagnostic-criteria-dsm-5/'

# Save Output

In [36]:
df.to_excel('Excel/Source3_1.xlsx',index=False)